In [1]:
from Resources.lib import *
from Resources import lib as lib
from Resources import model_package as MODEL

2024-02-09 17:08:35,776 numexpr.utils INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-02-09 17:08:35,778 numexpr.utils INFO NumExpr defaulting to 8 threads.
2024-02-09 17:08:37.570286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 17:08:37.570327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 17:08:37.571109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 17:08:37.578391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instruction

In [2]:
###----------------
### Some parametersimport warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
import tensorflow as tf
###----------------

inpDir = './upload'
outDir = '../DATA'
subDir = 'csvs'
audDir = "AUDIO"

RANDOM_STATE = 24               # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)    # Set Random Seed for reproducible  results

EPOCHS = 11                     # number of epochs
ALPHA = 0.001                   # learning rate
NUM_SAMPLES = 1280              # How many samples we want to generate 
NOISE = 0.2                     # Noise to be introduced in the data
TEST_SIZE = 0.2
BATCH_SIZE = 26
PATIENCE = 20

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [3]:
df = pd.read_csv("./DATA/csvs/Obama-to-Trump_FAKE.csv")
df = df.drop(columns="LABEL")
df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.318201,0.035020,3065.745194,2612.185396,6128.153853,0.153431,-307.85600,55.697018,-17.379436,6.981838,...,0.155245,-16.165169,5.405939,2.549457,-5.112835,-0.544781,-12.065888,-1.423916,-13.294633,-6.527376
1,0.485404,0.001413,2198.233964,2432.632174,4381.517445,0.105169,-500.68866,102.653366,-16.533148,17.381489,...,2.993131,-1.617074,6.602706,4.180027,1.369403,3.928801,-5.393010,-1.239054,-7.340294,1.440183
2,0.551631,0.028426,2064.519562,2241.802146,3640.334806,0.110873,-446.95303,68.518830,3.167167,3.967508,...,9.420251,-1.530157,-2.828733,2.390953,7.014152,7.069071,-3.359785,-2.954807,-4.916270,-2.155269
3,0.261755,0.064631,2105.320719,2339.191843,4132.172741,0.073464,-296.35320,81.210450,-3.549155,-9.961495,...,3.260442,-13.572388,-8.630247,-0.499472,-4.775998,2.591343,-13.655369,-4.985857,-10.444016,-8.177310
4,0.213219,0.080373,2662.924633,2729.530399,5498.307662,0.101196,-234.19562,47.477127,-14.037437,5.460664,...,-2.586541,-22.433916,-7.057376,0.092570,-5.931805,3.295625,-16.614754,-3.367248,-14.248423,-10.043166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.529265,0.000707,2028.525797,2479.323303,4375.155362,0.076749,-568.84840,107.816574,-5.137602,9.866554,...,2.630051,2.594823,-1.932425,0.252002,0.179408,0.435652,-3.669945,0.803377,-2.939240,-0.092321
596,0.325437,0.003105,1921.683521,2082.285528,3689.763295,0.093850,-453.04553,112.656230,-32.079224,-11.304557,...,9.616378,-5.593739,0.452518,5.767397,1.302976,-0.954953,-2.310495,5.190651,-5.392072,-3.328121
597,0.465283,0.007240,1762.895578,1671.399238,3117.420543,0.099432,-374.81183,119.899790,-63.249557,-26.146914,...,5.508694,-9.603080,3.404631,16.177050,-2.965148,-9.467863,-3.318303,4.828074,-6.467795,-0.744244
598,0.465095,0.008305,1951.899813,1802.785408,3309.751199,0.106434,-357.91193,97.499120,-83.554210,-29.018602,...,5.169014,-5.076159,-2.959461,4.662685,3.482278,-1.149203,-2.839118,8.388673,2.884281,-0.787594


In [4]:
model = MODEL.Model()
model.buid_model()
model.load_weights("Weights/demo_weigths")
result = model.predict(df)

2024-02-09 17:08:39.709109: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-09 17:08:39.783125: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-09 17:08:39.783708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-09 17:08:39.786516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-09 17:08:39.786882: I external/local_xla/xla/stream_executor

In [5]:
result

'Fake -  96.33%'

In [6]:
lib.TF_Predict(df)

2024-02-09 17:08:40,863 tensorflow WARNING Layer gru_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


'Fake -  96.33%'